<a href="https://colab.research.google.com/github/Balachandar-Ganesan/GraphRAG/blob/main/AgentAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewAI

In [ ]:
!pip install langchain_openai

In [5]:
from google.colab import userdata
import os
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
    # Then set it for the OpenAI library
import openai
#openai.api_key = openai_api_key
client = openai.OpenAI(api_key=openai_api_key)

In [5]:
from crewai import Agent

# Define a basic agent
general_agent = Agent(
    role='Movie  Agent',
    goal='Find Theaters to watch Movies requested by user with description',
    backstory='Need to find theaters where movies are screened.You can access information from https://in.bookmyshow.com/ and https://ticketnew.com/movies for schedule'
)

In [6]:
from crewai import Task

# Define a task for the agent
planning_task = Task(
    description='Find Theaters Near Cognizant Siruseri Chennai office where Tamil Movies are running currently',
    expected_output='List of Theaters and address with Movie Name,Next earliest Date where movie can be watched',
    agent=general_agent
)

In [7]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI
# Assemble the crew
crew = Crew(
    agents=[general_agent],
    tasks=[planning_task],
    process=Process.sequential,
    verbose=True
)
llm = ChatOpenAI(model="gpt-4-turbo-preview")


In [ ]:
result = crew.kickoff()

# Display the final output
print(result)

In [ ]:
!pip install 'crewai[tools]' tavily-python

In [7]:
from crewai_tools import TavilySearchTool

    # To search only on specific domains
tavily_tool_include = TavilySearchTool(
        include_domains=["https://in.bookmyshow.com/,https://ticketnew.com/movies/chennai"]
    )

    # To exclude specific domains from the search
tavily_tool_exclude = TavilySearchTool(
        exclude_domains=["spam-site.net", "old-blog.info"]
    )

In [ ]:
from crewai import Agent, Task, Crew
import os
os.environ["TAVILY_API_KEY"] = ""
MovieAgent = Agent(
        role='Movie  Agent',
        goal='Find Theaters to watch Movies requested by user with description.',
        backstory='Need to find theaters where movies are screened.You can access information from https://in.bookmyshow.com/ and https://ticketnew.com/movies for schedule.',
        tools=[tavily_tool_include], # Use the tool configured with include_domains
        verbose=True
    )

SearchTask = Task(
        description='Find Theaters Near Cognizant Siruseri Chennai office where Tamil Movies are running currently.',
        expected_output='List of Theaters and address with Movie Name,Review and Rating if possible ,Next earliest Date where movie can be watched in a CSV format',
        agent=MovieAgent
    )

crew = Crew(
        agents=[MovieAgent],
        tasks=[SearchTask],
        verbose=True
    )

result = crew.kickoff()
print(result)